In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import binary_cross_entropy_with_logits
from torch.optim import Adam

In [2]:
from transformers import ElectraTokenizer, TFElectraForSequenceClassification

2024-12-09 13:19:19.162407: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 13:19:19.170462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733750359.180364  398855 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733750359.183345  398855 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 13:19:19.193597: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [10]:
# 최종 증강 데이터 활용
train_data = pd.read_csv('/workspace/train_Augumented_data_preproc_last.csv')
train_data

,idx,conversation,label
0,0,야 새꺄 일로와 바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더...,2
1,1,민수야 나 수술비가 필요해서 그러는데 5만 원만 빌려줄 수 있을까? 응? 안돼. 나...,1
2,2,나 이거 할인해 줘 이 상품은 할인 상품이 아니세요. 무슨 소리야? 어제는 됐다고....,3
3,3,어때? 예쁜 손가락이 곧 없어질 텐데. 림 그리는 사람한테는 손이 생명이에요 그러게...,0
4,4,이빨을 못쓰게 뽑아버려야겠어 왜 그러십니까 왜긴 네 하는 짓거리가 거슬려서 참을 수...,0
...,...,...,...
9895,9895,"이봐, 당신은 만화가 보이십니까? 반환. 다시,이 아기는 너무 씹어 봐요.미래에, ...",3
9896,9896,"결혼식을 축하해 주셔서 감사합니다. 물론, 우리 동문은 가장 빨리 가고 있습니다. ...",3
9897,9897,"뜨겁다, 무슨 일이야?당신은 그 담배를 피우고있었습니다. 왜 당신은 횡단 보도이지만...",0
9898,9898,"좋아요, 당신의 일이 뭐야? 내 머리카락이 여기 나왔다.양육자 환불이 어렵습니다. ...",3


In [11]:
# 데이터 로드
data = train_data

class_mapping = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반 대화": 4
}

# 클래스 이름을 숫자로 변환
data['label'] = data['class'].map(class_mapping)
data = data.drop(columns=['class'])
data

KeyError: 'class'

In [ ]:
# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['conversation'], data['label'], test_size=0.2, random_state=42
)

In [ ]:
# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": label
        }


In [ ]:
# koelectra 모델 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=5,
    problem_type="multi_label_classification"
).to(device)

In [ ]:
# 데이터셋 생성
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

In [ ]:
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

In [ ]:
# 데이터 준비
texts = ["이 문장은 협박에 해당합니다.", "이 문장은 일반 문장입니다."]
labels = [[1, 0, 0, 0, 0], [0, 0, 0, 0, 1]]  # Multi-hot encoding

tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

train_dataset = TextDataset(texts, labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)



# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

In [ ]:
# 옵티마이저 및 손실 함수 설정
# 옵티마이저와 스케줄러 정의
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
num_training_steps = len(train_loader) * epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
from transformers import AutoModelForSequenceClassification, get_scheduler

# 하이퍼파라미터
epochs = 22
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

best_loss = float('inf')
checkpoint_path = "/workspace/koelectra_best_model_checkpoint_right1.pth"  # 체크포인트 저장 경로

# 학습 및 검증 루프
for epoch in range(epochs):
    # Training Loop
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()

        # 모델 출력 및 손실 계산
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # 역전파
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 정확도 계산
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    train_accuracies.append(correct / total)
    scheduler.step()  # 학습률 스케줄러 업데이트

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}")

    # Validation Loop
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_correct / val_total)

    print(f"Epoch {epoch + 1}/{epochs}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}")

    # Checkpoint 저장
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch + 1} with val loss {avg_val_loss:.4f}")

# Checkpoint 불러오기 함수
def load_checkpoint(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    best_loss = checkpoint['loss']
    print(f"Checkpoint loaded. Starting at epoch {start_epoch} with best loss {best_loss:.4f}")
    return model, optimizer, start_epoch, best_loss

# 체크포인트 불러오기 예시
# model, optimizer, start_epoch, best_loss = load_checkpoint(model, optimizer, checkpoint_path)

In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 그래프
plt.figure(figsize=(12, 6))

# Loss 그래프
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, marker='o', label="Training Loss")
plt.plot(range(1, epochs + 1), val_losses, marker='o', label="Validation Loss", color='red')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.grid(True)
plt.legend()

# Accuracy 그래프
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies, marker='o', label="Training Accuracy", color='orange')
plt.plot(range(1, epochs + 1), val_accuracies, marker='o', label="Validation Accuracy", color='green')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

### F1 score val 평가

In [ ]:
from sklearn.metrics import f1_score, classification_report
import torch

# 1. 평가 루프
model.eval()  # 모델 평가 모드 전환
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:  # 기존에 학습 루프에서 사용하던 val_loader 활용
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)  # 실제 라벨

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()  # 예측값
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())  # 실제값

# 2. F1 Score 및 Classification Report 계산
macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["협박", "갈취", "직장 내 괴롭힘", "기타 괴롭힘", "일반 대화"]))
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")

## 일반대화 제외 f1 score 계산

In [ ]:
from sklearn.metrics import f1_score, classification_report
import torch

# 제외할 클래스 정의 (예: '일반 대화' 클래스가 4번이라고 가정)
exclude_class = 4

# 1. 평가 루프
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# 2. 클래스 제외
filtered_preds = [pred for pred, label in zip(all_preds, all_labels) if label != exclude_class]
filtered_labels = [label for label in all_labels if label != exclude_class]

# 제외된 클래스를 반영한 레이블 정의
filtered_class_labels = [0, 1, 2, 3]  # 제외된 '일반 대화' 클래스 번호를 제외한 나머지 클래스
filtered_target_names = ["협박", "갈취", "직장 내 괴롭힘", "기타 괴롭힘"]  # 제외된 클래스 이름 제거

# 3. F1 Score 및 Classification Report 계산
macro_f1 = f1_score(filtered_labels, filtered_preds, average='macro')
weighted_f1 = f1_score(filtered_labels, filtered_preds, average='weighted')

print("Classification Report (Excluding '일반 대화'):")
print(classification_report(filtered_labels, filtered_preds, labels=filtered_class_labels, target_names=filtered_target_names))
print(f"Macro F1 Score (Excluding '일반 대화'): {macro_f1:.4f}")
print(f"Weighted F1 Score (Excluding '일반 대화'): {weighted_f1:.4f}")

### Test data 평가

In [ ]:
test_preproc_result2 = pd.read_csv('/workspace/test_data_preproc_last.csv')
# 욕설까지 필터링 된 test_data 사용
data = test_preproc_result2
data

In [ ]:
# 데이터 로드 및 전처리
data['text'] = data['text'].apply(lambda x: x.strip())

class_mapping = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반 대화": 4
}

In [ ]:
# 2. 테스트 데이터셋 생성
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=4096):  # KoBigBird는 4096 토큰까지 지원
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }

# 3. 모델 및 토크나이저 로드
model_path = "/workspace/koelectra_best_model_checkpoint_right1.pth"
model_name = "monologg/kobigbird-bert-base"

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

# 체크포인트 로드
checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 4. 데이터로더 생성
test_texts = test_data['text']  # test_data의 conversation 열 사용
test_dataset = TestDataset(test_texts, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=4)  # KoBigBird는 큰 모델이므로 작은 배치 크기 사용

# 5. 평가 루프
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(preds)


In [ ]:
# 6. 결과 저장
test_data['predicted_label'] = all_preds  # 예측 결과를 새로운 열로 추가
output_path = "/workspace/koelectra_best_model_checkpoint_right1.csv"  # 결과 저장 경로
test_data.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")
test_data
